Questions:
• Can you analyze the measurement data and give advice on which IPC’s could be downsized?
• Describe the steps that you have taken to get to the advice (please mention which data you have included and why).
• Do a general check on the data and outline where you see strange activity (and why).
• What else would be needed to run such an analysis continuously across all IPCs?

In [1]:
# Import all the packages needed

import csv
import pandas as pd


In [ ]:
# Specify the path to the CSV file
csv_path_file = "C:/Users/User/Desktop/Itility Test Case/dataset.csv"

# Read the CSV file uing a pandas DataFrame
df = pd.read_csv(csv_path_file)

# print the DataFrame for information
print(df)

In [ ]:
# Initialize an empty list to store the data
data = []

# Open the CSV file and read its contents using DictReader
with open(csv_path_file, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Loop through each row in the CSV and append it to the data list
    for row in csv_reader:
        data.append(row)

# Print the data
for row in data:
    print(row)

In [ ]:
# Accessing a specific column by its header name
column_data = df['IPC']
print(column_data)

In [4]:
# Count the number of distinct values in the specified column
distinct_ipc_values = df['IPC'].nunique()

# Print the number of distinct values in the column
print("Number of distinct IPC values:", distinct_ipc_values)

Number of distinct IPC values: 4261


In [8]:
# Check if there are any duplicate entries
dup_rows = df[df.duplicated()]
print("The number of duplicated rows is", dup_rows)

The number of duplicated rows is Empty DataFrame
Columns: [IPC, Data Factory, time, AvgValue, MinValue, MaxValue, MetricId, MemoryMB, CpuMHz, NumCpu, Unnamed: 10, Unnamed: 11, Unnamed: 12]
Index: []
